In [ ]:
#Required imports
import warnings
warnings.filterwarnings("ignore")
import os
import math
import numpy as np
import random
import pandas as pd
import pyfolio as pf
import talib
from pylab import plt, mpl
import datetime as dt
import tensorflow as tf
from keras.layers import Dense, Dropout, BatchNormalization,Conv1D,Flatten,MaxPooling1D,LSTM
from keras.models import Sequential
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, TimeSeriesSplit
from pandas_datareader import DataReader
from datetime import datetime
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#Notebook display imports
from IPython.display import set_matplotlib_formats
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', '{:.4f}'.format)
np.set_printoptions(suppress=True, precision=4)
set_matplotlib_formats('retina')
%config InlineBackend.figure_format = 'svg'

random.seed(50) #for reproducibility

### Getting Data and Preparing Data for Modeling

In [ ]:
start_date=datetime(1999, 1, 1)
end_date=datetime(2020,9,30)
df = DataReader('AAPL',  'yahoo', start=start_date, end=end_date)
df.drop("Adj Close",axis=1,inplace=True)
df.tail()

We want to use the first trading of each month as a feature of our model.

In [ ]:
start_year=start_date.year
start_month=start_date.month
end_year=end_date.year
end_month=end_date.month

first_days=[]
# First year
for month in range(start_month,13):
    first_days.append(min(df[str(start_year)+"-"+str(month)].index))
# Other years
for year in range(start_year+1,end_year):
    for month in range(1,13):
        first_days.append(min(df[str(year)+"-"+str(month)].index))
# Last year
for month in range(1,end_month+1):
    first_days.append(min(df[str(end_year)+"-"+str(month)].index))

For each month we need the means of the month, the first trading day of the current month (and its open price), and the first trading day of the next month (and its open price): our models will predict based on these data.

The feature *quot* is the quotient between the open price of the first trading day of the next month and the open price of the first trading day of the current month. This will help capture the variability of the stock price from month to month

We also add one year and 2 years moving average to capture some momentum.

In [ ]:
def model_data(df):

    dfm=df.resample("M").mean()
    dfm=dfm[:-1] # As we said, we do not consider the month of end_date
    
    dfm["fd_cm"]=first_days[:-1]
    dfm["fd_nm"]=first_days[1:]
    dfm["fd_cm_open"]=np.array(df.loc[first_days[:-1],"Open"])
    dfm["fd_nm_open"]=np.array(df.loc[first_days[1:],"Open"])
    dfm["quot"]=dfm["fd_nm_open"].divide(dfm["fd_cm_open"])
    
    dfm["mv_avg_12"]= dfm["Open"].rolling(window=12).mean().shift(1)
    dfm["mv_avg_24"]= dfm["Open"].rolling(window=24).mean().shift(1)
    
    dfm.dropna(inplace=True)
    
    return dfm

In [ ]:
dfm=model_data(df)
dfm.head()
dfm.tail()

#### Helper Functions to calculate the Strategy returns

In [ ]:
def yield_gross(df:pd.DataFrame,v: np.array)-> tuple:
    """
    :param df: dataframe used to model the strategy
    :param v: array representiong days in or out of the market 1 for day in and o for day out. The length of the array is the duration of the strategy
    :returns (cum_return, average_annual return) tuple of cumulative return and average annual return over the strategy period
    """
    #cumulative return over the period +1
    prod=(v*df["quot"]+1-v).prod()
    n_years=len(v)/12
    return (prod-1)*100,((prod**(1/n_years))-1)*100

The following function will be used to compute the net yield.

Given any vector of zeros and ones as input, *separate_ones* will return the sequence of vectors of groups of adjacent ones and a scalar equal to the number of groups of adjacent ones.

In [ ]:
def separate_ones(u: np.array)-> tuple:
    """
    :param u: array representiong days in or out of the market 1 for day in and o for day out. The length of the array is the duration of the strategy
    :returns (out, n): a tuple of a numpy array where each row represents the number of consecutive days in the market and an integer that represents the 
    total of in and out out of the market
    """
    
    u_ = np.r_[0,u,0]
    index = np.flatnonzero(u_[:-1] != u_[1:])
    v,w = index[::2],index[1::2]
    if len(v)==0:
        return np.zeros(len(u)),0
    
    n,m = len(v),len(u)
    o = np.zeros(n*m,dtype=int)

    r = np.arange(n)*m
    o[v+r] = 1

    if w[-1] == m:
        o[w[:-1]+r[:-1]] = -1
    else:
        o[w+r] -= 1

    out = o.cumsum().reshape(n,-1)
    return out,n

In [ ]:
u=np.array([0,1,1,0,1,1,1,0,1])
separate_ones(u)

In [ ]:
def roll_window(data:pd.DataFrame, window_size:int = 1)-> pd.DataFrame:
    '''
    :param data: dataframe to be used by the model
    :param window: this is the look back period on which the LSTM model will train on
    :returns data: transformed dataframe including the chosen rolling windows
    
    '''
    data_s = data.copy()
    for i in range(window_size):
        data = pd.concat([data, data_s.shift(-(i + 1))], axis = 1)
        
    data.dropna(axis=0, inplace=True)
    return(data)

In [ ]:
def data_to_model(dfm: pd.DataFrame)-> tuple:
    '''
    :param data: dataframe to be used by the model
    :returns (X: np.array, y:np.array): A tuple for the features and targets used for the model
    '''
    #Standardize the data
    scaler=MinMaxScaler(feature_range=(0,1))
    dg=pd.DataFrame(scaler.fit_transform(dfm[["High","Low","Open","Close","Volume","fd_cm_open",\
                                          "mv_avg_12","mv_avg_24","fd_nm_open"]].values))
    X=dg[[0,1,2,3,4,5,6,7]]
    X=roll_window(X,window)
    X=np.reshape(X.values,(X.shape[0],window+1,8))
    
    y=np.array(dg[8][window:])
    
    return X,y

### Creation of training and testing samples and model creation

In [ ]:
window=2
X,y=data_to_model(dfm)
print(X.shape,y.shape)
mtest=60 # number of samples to test
X_train=X[:-mtest-1,:,:]
X_test=X[-mtest-1:,:,:]
y_train=y[:-mtest-1]
y_test=y[-mtest-1:]

In [ ]:
def model_lstm(window,features):
    
    model=Sequential()
    model.add(LSTM(300, input_shape = (window,features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(200,  return_sequences=False)) # there is no need to specify input_shape here
    model.add(Dropout(0.5))
    model.add(Dense(100,kernel_initializer='uniform',activation='relu'))        
    model.add(Dense(1,kernel_initializer='uniform',activation='relu'))
    model.compile(loss='mse',optimizer=Adam(lr=0.001))
    
    
    return model

In [ ]:
model_lstm=model_lstm(window+1,8)
print(model_lstm.summary())

Let's use a callback that will reduce the training rate if training reaches a plateau and stopped progressing

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=25, verbose=1,\
                                                 factor=0.25, min_lr=0.00001)

In [ ]:
history_lstm=model_lstm.fit(X_train,y_train,epochs=500, batch_size=24, validation_data=(X_test, y_test), \
                  verbose=1, callbacks=[learning_rate_reduction, EarlyStopping(monitor='loss', restore_best_weights=True, patience=5)],shuffle=False)

Let's plot training and validation loss

In [ ]:
plt.plot(history_lstm.history['loss']);
plt.plot(history_lstm.history['val_loss']);
plt.title('model loss');
plt.ylabel('loss');
plt.xlabel('epoch');
plt.legend(['train', 'test'], loc='upper right');
plt.show();

In [ ]:
model_lstm.save_weights("lstm_weights.h5")

Let's evaluate the model

In [ ]:
model_lstm.evaluate(X_test, y_test)

### Forecasting

In [ ]:
y_pred_train_lstm=model_lstm.predict(X_train)

In [ ]:
plt.figure(figsize=(8,6));
plt.plot(y_train, label="actual");
plt.plot(y_pred_train_lstm, label="prediction by lstm model");
plt.legend(fontsize=10);
plt.grid(axis="both");
plt.title("Actual open price and pedicted one on train set",fontsize=15);
plt.show();

### Trading Strategy

As long as the predicted price for the next month is greater than the current price, we stay long, otherwise we'll short.The vectors v represents the "in months" (as 1s) and "out months" (as 0s)

In [ ]:
y_pred_lstm=model_lstm.predict(X_test)

In [ ]:
w_lstm=np.diff(y_pred_lstm.reshape(y_pred_lstm.shape[0]),1)
v_lstm=np.maximum(np.sign(w_lstm),0)

In [ ]:
plt.figure(figsize=(8,6));
plt.plot(y_test, label="actual");
plt.plot(y_pred_lstm, label="prediction lstm");
plt.plot(v_lstm,label="In and out lstm");
plt.legend(fontsize=10);
plt.grid(axis="both");
plt.title("Actual open price, predicted ones and vectors on in and out moments",fontsize=15);
plt.show();

Now we can compare our deep learning trading strategy with the buy and hold strategy. In order to do so, we compute the corresponding vectors *v_bh*, which selects the months in which we are going to stay in the market.

In [ ]:
test=dfm.iloc[-mtest:,:] 
v_bh=np.ones(test.shape[0])

In [ ]:
def gross_portfolio(df,w):
    portfolio=[ (w*df["quot"]+(1-w))[:i].prod() for i in range(len(w))]
    return portfolio

In [ ]:
plt.figure(figsize=(8,6));
plt.plot(gross_portfolio(test,v_bh),label="Portfolio Buy and Hold");
plt.plot(gross_portfolio(test,v_lstm),label="Portfolio LSTM");
plt.legend(fontsize=10);
plt.grid(axis="both");
plt.title("Gross portfolios of the strategy", fontsize=15);
plt.show();

### Performance Analysis

In order to perform our performance analysis with Pyfolio, we need to create a dataframe with the predicted returns by the model

In [ ]:
print("Test period of {:.2f} years, from {} to {} \n".format(len(v_bh)/12,str(test.loc[test.index[0],"fd_cm"])[:10],\
      str(test.loc[test.index[-1],"fd_nm"])[:10]))

results=pd.DataFrame({})
results["Method"]=["Buy and hold","LSTM"]

vs=[v_bh,v_lstm]
results["Total gross yield"]=[str(round(yield_gross(test,vi)[0],2))+" %" for vi in vs]
results["Annual gross yield"]=[str(round(yield_gross(test,vi)[1],2))+" %" for vi in vs]
results

### Conclusion and ways to improve the model

Although our strategy did not beat the benchmark, it did a satisfactory job of earning a compound annual growth rate(CAGR) of 24.1% which is a very good performance. There are several ways, this model could be improve to earn an even higher CAGR. 
The first one could have been to break the sample periods into at least 2 or 3 periods because of the structural breaks that occures in 2008-2009( The financial crisis) and the Covid19 pandemic which started in early 2020. We could have then produced one model for each period and concatenate the 3 models to serve as a combined model for our prediction.
The second idea could have been to model a two inputs/two outputs model where one input will serve for sentiment analysis and the second input will serve to determine the price of the stock. Then we will only decide whether we are in and out of the market based only on the price direction but also on the sentiment.
Finally for all models, we could have performed hyperparameter tuning by performing RandomizedSearchCV to identify the best parmeters.

#### References
- Advanced Deep Learning with Keras, Zach Deane Meyer, DataCamp,  2019
- Machine Learning in Finance,  Nathan George, DataCamp, 2019
- Artificial Intelligence in Finance, Yves Hillpsich, Oreilly Media, October 2020
- Algorithmic Trading with Keras, Federicko Woelenski, Oreilly Media, 2018
